# Exploring profitable free iOS & Android Apps 

**What the project is about?**
-I will be finding free applications that generate a lot of traffic. This data can help future developer understand, what type apps attracting the most users.

**What your goal is in this project?**
-This guided project is meant to clean up the data. The reason for cleaning it up is so, we only pull out the revelant data we need. Revelant data will help us create a hypothese.

In [ ]:
# import for googleplaystore

from csv import reader
opened_file = open('googleplaystore.csv',encoding="utf-8")
read_file = reader(opened_file)
android_all = list(read_file)
android_header = android_all[0]
android_data = android_all[1:]

# print(android_header)
# print(anandroid_all)

In [ ]:
# import for Applestore

from csv import reader
opened_file = open('AppleStore.csv',encoding="utf-8")
read_file = reader(opened_file)
ios_all = list(read_file)
ios_header = ios_all[0]
ios_data = ios_all[1:]

# print(ios_header)
# print(ios_data)


In [ ]:
# explore_date function to figure out shape or size of the conver

def explore_data(data, start, end, row_and_column=False):
    data_slice = data[start: end]
    
    if row_and_column:
        row_length = len(data)
        column_length = len(data[0])
        return row_length, column_length
    
    return data_slice


In [ ]:
# explore Apple Appstore data set

result = explore_data(ios_data, 2001, 2005, row_and_column=False)
print(result)

row_column_ios = explore_data(ios_data, 2001, 2005, row_and_column=True)
print(row_column_ios)
row, column = row_column_ios

print("\n")
print("row:", row)
print("column:", column)

In [ ]:
# explore Google Playstore data set

print(explore_data(android_data, 2001, 2005))
row_column_android = explore_data(android_data, 2001, 2005, row_and_column=True)
row, column = row_column_android

print("\n")
print("row:", row)
print("column:", column)

In [ ]:
# remove dataset with missing columns 

def remove_less(data, data_header):
    for row in data:
        if len(row) < len(data_header):
            print(row)
            print(len(row))
            print(data.index(row))
            del data[data.index(row)]
            
remove_less(ios_data, ios_header)            
remove_less(android_data, android_header)

print("ios number of row:",len(ios_data))
print("android number of row:",len(android_data))

In [ ]:
# quick way of finding duplicates in our android_data by name

for app in android_data:
    app_name = app[0]
    if app_name == "Quick PDF Scanner + OCR FREE":
        print(app)

In [ ]:
# Finding which andriod apps have duplicates and listing some

unique_apps = []
duplicate_apps = []

for app in android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print("Number of duplicate apps is", len(duplicate_apps), "\n")
print("Number of unique apps is", len(unique_apps), "\n")
print("Some duplicate:", duplicate_apps[:20])

In [ ]:
# dictionary with key=unique app name, and value=highest number of reviews of each app

reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print("Andriod expected length:", len(reviews_max))
print("Andriod expected length after removing the duplicate:", len(android_data) - len(duplicate_apps))
# print(reviews_max)

In [ ]:
# pick recent review and clean duplicates 

android_data_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_data_clean.append(app)
        already_added.append(name)

explore_data(android_data_clean, 0, 3, row_and_column=True)

In [ ]:
# for android apps 

print(android_data_clean[4412][0])
print(android_data_clean[7940][0], "\n")

# for iOS apps 

print(ios_data[813][1])
print(ios_data[6731][1])

In [ ]:
# char who pass 127 on ASCI table are not english. More than 3 outside 0-127 = not eng app

def is_eng(string):
    count = 0
    for char in string:
        if ord(char) >= 127:
            count += 1
            if count > 3:
                return False
    return True

eng_or_not = "Is this string english?:"
print(eng_or_not, is_eng("Docs To Go™ Free Office Suite"))
print(eng_or_not, is_eng("Instachat 😜"))
print(eng_or_not, is_eng("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(eng_or_not, is_eng("™™™™"))   

In [ ]:
# english_apps function to clean both ios and andriod apps that aren't english

def english_apps(data_set, ios=False):
    english_apps_data = []
    
    for app in data_set:
        app_name_android = app[0]
        app_name_ios = app[1]
        
        if not ios:
            if is_eng(app_name_android):
                english_apps_data.append(app)
                
        if ios:
            if is_eng(app_name_ios):
                english_apps_data.append(app)
    
    return english_apps_data

android_data_clean_english = english_apps(data_set=android_data_clean)
ios_data_english = english_apps(data_set=ios_data, ios=True)


print(explore_data(android_data_clean_english, 0, 3, True))
print("\n")
print(explore_data(ios_data_english, 0, 3, True))

In [ ]:
# free_apps function created to strip paid apps, which determines my final length. 

def free_apps(data_set, ios=False):
    free_apps_data = []
    
    for app in data_set:
        price_andrioid = float(app[7].strip("$"))
        
        if not ios:
            if price_andrioid ==0:
                free_apps_data.append(app)
                
        if ios:
            price_ios = float(app[4])
            if price_ios == 0:
                free_apps_data.append(app)
                
    return free_apps_data

android_data_final = free_apps(data_set=android_data_clean_english)
print("Final length of android data after cleaning paid apps:", len(android_data_final))

ios_data_final = free_apps(data_set=ios_data_english, ios=True)
print("Final length of ios data after cleaning paid apps:", len(ios_data_final))

In [ ]:
def freq_table(data_set, index):
    table = {}
    total = 0
    
    for row in data_set:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key]/total) *100
        table_percentages[key] = percentage 
        
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [ ]:
# ios prime genre 

display_table(ios_data_final, -5)

In [ ]:
# Andriod Category 

display_table(android_data_final, 1) 

In [ ]:
# android genres 

display_table(android_data_final, -4) 

# iOS Prime_genre analysis 

**What is the most common genre? What is the runner-up?**

-The most common is genre is games at 58.16%. The runner up is the Entertainment apps at 8%.

**What other patterns do you see?**

-Games, entertainment, social media, music, etc. Seems, people like using apple devices for their entertainment mostly.

**What is the general impression — are most of the apps designed for practical purposes (education, shopping, utilities, productivity, lifestyle) or more for entertainment (games, photo and video, social networking, sports, music)?**

Most of the apps being made for iOS are being made for peoples entertainment.

**Can you recommend an app profile for the App Store market based on this frequency table alone? If there's a large number of apps for a particular genre, does that also imply that apps of that genre generally have a large number of users?**

-I would recommend the Socail Networking genre. Since, game apps just saturared and I know people are more glued to the Soocial network genre. 


# Android category and genres analysis

**What are the most common genres?**
-The Google play store most common are family,tools,lifestyle,etc) 

**What other patterns do you see?**
-I notice that the family Category is the highest at 18.907 and this is where most of the games are for kids. But seems like Android users overall are most divese compared to apple users.

**Compare the patterns you see for the Google Play market with those you saw for the App Store market.**
-I belive the Google market is less for like entertainment and more for bussiness, lifestyle, productivity, etc. This tell me that their crowd might be more mature compared to apple users.

**Can you recommend an app profile based on what you found so far? Do the frequency tables you generated reveal the most frequent app genres or what genres have the most users?**
-Yes I would recommened something that will be helpful to android users. So, that would fall under the Tools genre. The frequent table shows us how often apps are downloaded in that catagory. For example "Navigation" is really high because everyone uses apps like Waze and Google Maps.



In [ ]:
# find most popular apps by genre using average number of installs (ios)

genres_ios = freq_table(ios_data_final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_data_final:
        genre_app = app[-5]
        if genre_app == genre:
            n_reviews = float(app[5])
            total += n_reviews
            len_genre += 1
    avg_num_ratings = total / len_genre
    print(genre, ":", avg_num_ratings)

In [ ]:
# ios apps that contribute to "Navigation" being one of the top apps genre 

for app in ios_data_final:
    if app[-5] == "Navigation":
        print(app[1], ":", app[5], )

In [ ]:
# ios apps that contribute to "Social Networking" being one of the top apps genre 

for app in ios_data_final:
    if app[-5] == "Social Networking":
        print(app[1], ":", app[5], )

In [ ]:
# data doesn't work the same as it did with iOS so we have to clean it up
display_table(android_data_final, 5)

In [ ]:
# clean up data and converting data by using float to removing unwanted strings
categories_andoid = freq_table(android_data_final, 1)

for category in categories_andoid:
    total = 0
    len_category = 0
    for app in android_data_final:
        category_app = app[1]
        
        if category_app == category:
            num_installs = app[5]
            num_installs = num_installs.replace(",", "")
            num_installs = num_installs.replace("+", "")
            total += float(num_installs)
            len_category += 1
            
    avg_num_installs = total / len_category
    print(category, ":", avg_num_installs)

In [ ]:
for app in android_data_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

## Overall analysis

*I personally recommend an app in the communcaiton catagory. This is because I feel people are most attached to these app and can't get enough of them. If done correctly it will be explosive. It might require great amount of work and time to build one but the market needs something that stands out from the rest.In total it has the most installs of any category. The communication category does well no matter the operating system as well.  